In [ ]:
import dataset, net, utils, loss

import torch, random, torchvision, importlib
from PIL import ImageDraw
from IPython.display import display
from tqdm import tqdm

torch.set_printoptions(linewidth=240, threshold=10000, sci_mode=False, precision=3)


Load Dataset

In [ ]:
importlib.reload(dataset)
importlib.reload(utils)

cv = utils.Converter()
rd = dataset.RisikoDataset(dataset_dir='Datasets/Generated_Dataset/train', cv=cv, is_trainset=True)
#rd = dataset.RisikoDataset(dataset_dir='Datasets/Professor_Material/real_images/', cv=cv, is_trainset=True, labels_extension='.txt')

#print(rd.loss_labels[0][rd.loss_labels[0][...,0]==1])


Find mean and std of full dataset

In [ ]:
cumulative_mean, cumulative_std = torch.zeros(3, dtype=torch.float32), torch.zeros(3, dtype=torch.float32)
pbar = tqdm(total=len(rd), unit='images')

for i in range(len(rd)):
    img = rd.images[i].type(torch.float32)
    torch.add(cumulative_mean, torch.mean(img, dim=(1,2)), out=cumulative_mean)
    torch.add(cumulative_std, torch.std(img, dim=(1,2)), out=cumulative_std)
    pbar.update(1)
pbar.close()

torch.divide(cumulative_mean, len(rd), out=cumulative_mean)
torch.divide(cumulative_std, len(rd), out=cumulative_std)

print('mean = ' + str(cumulative_mean))
print('std  = ' + str(cumulative_std))

In [ ]:
print(cv.grids_shape)
print(cv.grids_relative_area_sqrt)
print(cv.netout_grid_limits)
print(cv.netout_abox_multiplier[-1], cv.netout_abox_offset[-1])

Test Dataset and Conversion of labels

In [ ]:
lbl_max = torch.zeros((4,), dtype=cv.netout_dtype)
lbl_min = torch.full((4,), 10e30, dtype=cv.netout_dtype)
for lbl in rd.loss_labels:
    lbl = lbl[lbl[:,0] == 1][:,(3,4,6,7)]
    
    if lbl.shape[0] > 0:
        lbl_curr_max = lbl.max(dim=0)[0]
        lbl_curr_min = lbl.min(dim=0)[0]

        #print('current: max', lbl_curr_max, '\t min', lbl_curr_min)

        torch.maximum(lbl_max, lbl_curr_max, out=lbl_max)
        torch.minimum(lbl_min, lbl_curr_min, out=lbl_min)

print('MAX =', lbl_max)
print('MIN =', lbl_min)

In [ ]:
i = 0
print(rd.loss_labels[i][rd.loss_labels[i][:,0] == 1][:, 5])
print(torch.arange(rd.loss_labels[i].shape[0])[rd.loss_labels[i][:,0] == 1])

In [ ]:
colors = ( "blue", "red", "yellow", "purple", "black", "green")

rd.salt_pepper_hval = 150

rnd_idx = random.randint(a=0, b=rd.__len__()-1)
# rnd_idx = 37
print('Chosen image idx: ' + str(rnd_idx))
img_tensor, loss_labels = rd[rnd_idx]
# print(img_tensor)
img_tensor = rd.invnorm(img_tensor).type(torch.uint8)
# print(img_tensor)

labels = rd.cv.convert_losslabels_to_labels(loss_labels)
labels_origin = rd.labels[rnd_idx].clone()
rd.cv.convert_labels_from_relative_to_absolute_values(labels)
rd.cv.convert_labels_from_relative_to_absolute_values(labels_origin)
boxes = rd.cv.convert_boxes_cxcywh_to_x1y1x2y2(labels[:,1:])
print(boxes)

print('Number of objects: ' + str(boxes.shape[0]))

to_pil = torchvision.transforms.ToPILImage()
img = to_pil(img_tensor)

if (boxes.shape[0] != 0):

    if len(boxes.shape) == 1:
        boxes = boxes.unsqueeze(1)

    img_draw = ImageDraw.Draw(img)

    for i in range(boxes.shape[0]):

        x0, y0, x1, y1 = boxes[i]
        
        img_draw.point(labels[i,1:3].tolist(), fill='red')
        img_draw.rectangle((x0, y0, x1, y1), outline=colors[int(labels[i,0]%6)])

display(img)

In [ ]:
print(len(rd))
print(rd.loss_labels[0][rd.loss_labels[0][:,0] == 1])

Network forward test

In [ ]:
importlib.reload(net)

my_net = net.GridNet(abox_count=cv.abox_count)

rnd_idx = random.randint(a=0, b=rd.__len__()-1)
# rnd_idx = 313
print('Chosen index = ' + str(rnd_idx))
img_tensor, netout_labels = rd[rnd_idx]
print(netout_labels.shape)
img_tensor = img_tensor.type(cv.netout_dtype).unsqueeze(0)

my_net.train()
netout = my_net.forward(img_tensor)
print('Train shape: ' + str(netout.shape))
my_net.eval()
netout = my_net.forward(img_tensor)
print('Eval shape: ' + str(netout.shape))

In [ ]:
from torchinfo import summary

print(summary(my_net, input_size=(1,3,512,512)))
print(my_net)

Loss test

In [ ]:
importlib.reload(loss)

my_loss = loss.RisikoLoss(
    lambda_abox=0.1,
    lambda_coord=1,
    lambda_no_obj=0.05,
    lambda_class_obj=1,
    lambda_class_color=0.1,
    abox_count=cv.abox_count
)

artificial_labels = torch.zeros(size=(3,10,8), dtype=cv.netout_dtype)
artificial_labels[0,1] = torch.tensor((1,1,2,0.1,0.1,0,0.3,-0.2))
artificial_labels[0,2] = torch.tensor((1,1,5,0.1,0.1,1,0.3,-0.2))
artificial_labels[1,0] = torch.tensor((1,1,3,0.1,0.1,0,0.3,-0.2))

artificial_netout = torch.zeros(size=(3,10,19), dtype=cv.netout_dtype)
artificial_netout[0,1] = torch.tensor((1,1,0,0,10,0,0,0,0.1,0.1,10,0,0,0.3,-0.2,0,0,0,0))
artificial_netout[0,2] = torch.tensor((1,1,0,0,0,0,0,10,0.1,0.1,0,10,0,0,0,0.3,-0.2,0,0))
artificial_netout[1,0] = torch.tensor((1,1,0,0,0,10,0,0,0.1,0.1,10,0,0,0.3,-0.2,0,0,0,0))

print(my_loss(artificial_netout, artificial_labels))
